# Módulos

In [1]:
import numpy as np
import pandas as pd

# __Execução__

## Importar dados

In [2]:
# Definir caminhos de dados
path_enem_19 = 'dados/extracao/enem_2019_sg_uf_prova_ba.csv' 
path_enem_20 = 'dados/extracao/enem_2020_sg_uf_prova_ba.csv' 
path_enem_21 = 'dados/extracao/enem_2021_sg_uf_prova_ba.csv'
path_mdad_19 = 'dados/microdados/dicionarios_tratados/Dicionário_Microdados_Enem_2019.xlsx'
path_mdad_20 = 'dados/microdados/dicionarios_tratados/Dicionário_Microdados_Enem_2020.xlsx'
path_mdad_21 = 'dados/microdados/dicionarios_tratados/Dicionário_Microdados_Enem_2021.xlsx'

In [3]:
# Extrair bases completas de estudantes que realizaram a prova na bahia
df_e19 = pd.read_csv(path_enem_19, low_memory=False, sep=',', encoding='latin-1')
df_e20 = pd.read_csv(path_enem_20, low_memory=False, sep=',', encoding='latin-1')
df_e21 = pd.read_csv(path_enem_21, low_memory=False, sep=',', encoding='latin-1')

In [4]:
# Extrair dicionários de microdados
df_md_19 = pd.read_excel(path_mdad_19, sheet_name='DEPARA').fillna(method='ffill')
df_md_20 = pd.read_excel(path_mdad_20, sheet_name='DEPARA').fillna(method='ffill')
df_md_21 = pd.read_excel(path_mdad_21, sheet_name='DEPARA').fillna(method='ffill')

In [5]:
# Definir colunas ignoradas para nossa análise inicialmente
cols_to_drop = ['CO_UF_PROVA','SG_UF_PROVA',
                'TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT',
                'NU_NOTA_COMP1','NU_NOTA_COMP2','NU_NOTA_COMP3','NU_NOTA_COMP4','NU_NOTA_COMP5',
                'TX_GABARITO_CN','TX_GABARITO_CH','TX_GABARITO_LC','TX_GABARITO_MT','TX_RESPOSTAS_CN']

In [6]:
# Unificar bases de dados
df_enem = pd.concat([df_e19, df_e20, df_e21], axis=0)

# Deletar bases anteriores
del df_e19, df_e20, df_e21

In [7]:
# Remover colunas não avaliadas
df_enem.drop(columns=cols_to_drop, inplace=True)

## Tratar colunas

- Código pode trazer informações de defiência
- Trazer detalhe das colunas aqui
- Filtrar

### Aplicar dicionário para tornar valores mascarados em corretos

In [8]:
# Loop para aplicar devidamente cada dicionário ao ano respectivo
for ano, df_md in zip(sorted(df_enem['NU_ANO'].unique()), [df_md_19, df_md_20, df_md_21]):
    print(ano)
    # Definir condição de ano
    cond_ano = df_enem['NU_ANO'] == ano
    # Loop nas colunas que estão presentes no dicionário
    for col in [c for c in df_enem.columns if c in df_md['COLUNA'].unique()]:
        # Criar dicionário de para
        dp = dict(zip(df_md.loc[df_md['COLUNA']==col, 'VALOR_ATUAL'], df_md.loc[df_md['COLUNA']==col, 'VALOR_CORRETO']))
        # Aplicar
        df_enem.loc[cond_ano, col] = df_enem.loc[cond_ano, col].replace(dp)

2019
2020
2021


In [9]:
# Visualizar cada categoria para cada variável categórica
#for col in df_enem.columns[df_enem.dtypes == 'object']:
#    print(f'###################################### {col}')
#    vals = ' | '.join([str(c) for c in df_enem[col].unique()])
#    print(f'{vals}')

### Criar novas colunas

**Criar nova coluna - `EH_CAPITAL`**

In [10]:
# Criar novas colunas
df_enem['EH_CAPITAL_MUNICIPIO_PROVA'] = (df_enem['NO_MUNICIPIO_PROVA'] == 'Salvador').astype(int)
df_enem['EH_CAPITAL_MUNICIPIO_ESCOLA'] = (df_enem['NO_MUNICIPIO_ESC'] == 'Salvador').astype(int)

**Criar nova coluna - `EH_PROVA_DEFICIENCIA`**

In [11]:
# Definir dicionário com códigos que apontem para este tipo de prova:
dp_defi = {
           2019:{'CO_PROVA_CN':[519,523],
                 'CO_PROVA_CH':[520,524],
                 'CO_PROVA_LC':[521,525],
                 'CO_PROVA_MT':[522,526]},
           2020:{'CO_PROVA_CN':[604,605,684],
                 'CO_PROVA_CH':[574,575,654],
                 'CO_PROVA_LC':[584,585,664],
                 'CO_PROVA_MT':[594,595,674]},
           2021:{'CO_PROVA_CN':[915,916,917,996],
                 'CO_PROVA_CH':[885,886,887,996],
                 'CO_PROVA_LC':[895,896,897,976],
                 'CO_PROVA_MT':[905,906,907,986]}
          }

In [12]:
# Criar nova coluna
for ano, cols_lista in dp_defi.items():
    # Gerar condições
    c0 = df_enem['NU_ANO'] == ano
    c1 = df_enem['CO_PROVA_CN'].isin(cols_lista['CO_PROVA_CN'])
    c2 = df_enem['CO_PROVA_CH'].isin(cols_lista['CO_PROVA_CH'])
    c3 = df_enem['CO_PROVA_LC'].isin(cols_lista['CO_PROVA_LC'])
    c4 = df_enem['CO_PROVA_MT'].isin(cols_lista['CO_PROVA_MT'])
    # Montar coluna para o ano
    df_enem[f'EH_PROVA_DEFICIENCIA_{str(ano)}'] = (c0 & (c1 | c2 | c3 | c4)).astype(int)

In [13]:
# Unificar e binarizar
cols_temp = [c for c in df_enem.columns if 'EH_PROVA_DEFICIENCIA' in c]
df_enem['EH_PROVA_DEFICIENCIA'] = (df_enem[cols_temp].sum(axis=1) >= 1).astype(int)
df_enem.drop(columns=cols_temp, inplace=True)

**Criar nova coluna - `EH_PROVA_DIGITAL`**

In [14]:
# Definir dicionário com códigos que apontem para este tipo de prova
dp_digi = {
           2020:{'CO_PROVA_CN':[699,700,701,702],
                 'CO_PROVA_CH':[687,688,689,690],
                 'CO_PROVA_LC':[691,692,693,694],
                 'CO_PROVA_MT':[695,696,697,698]},
           2021:{'CO_PROVA_CN':[1011,1012,1013,1014], 
                 'CO_PROVA_CH':[999,1000,1001,1002],
                 'CO_PROVA_LC':[1003,1004,1005,1006],
                 'CO_PROVA_MT':[1007,1008,1009,1010]}
          }

In [15]:
# Criar nova coluna
for ano, cols_lista in dp_digi.items():
    # Gerar condições
    c0 = df_enem['NU_ANO'] == ano
    c1 = df_enem['CO_PROVA_CN'].isin(cols_lista['CO_PROVA_CN'])
    c2 = df_enem['CO_PROVA_CH'].isin(cols_lista['CO_PROVA_CH'])
    c3 = df_enem['CO_PROVA_LC'].isin(cols_lista['CO_PROVA_LC'])
    c4 = df_enem['CO_PROVA_MT'].isin(cols_lista['CO_PROVA_MT'])
    # Montar coluna para o ano
    df_enem[f'EH_PROVA_DIGITAL_{str(ano)}'] = (c0 & (c1 | c2 | c3 | c4)).astype(int)
# Unificar e binarizar
cols_temp = [c for c in df_enem.columns if 'EH_PROVA_DIGITAL' in c]
df_enem['EH_PROVA_DIGITAL'] = (df_enem[cols_temp].sum(axis=1) >= 1).astype(int)
df_enem.drop(columns=cols_temp, inplace=True)

**Criar nova coluna - Renda Familiar unificada através dos anos em `RENDA_FAMILIAR`**

In [ ]:
# Unificação de categorias de renda através dos anos
dp_renda = {'2019':{'Nenhuma Renda': ['Nenhuma renda.'],
                    'Até 1 salário mínimo': ['Até R$ 998,00.'],
                    'De 1 Salário Mínimo até 3 Salários Mínimos': ['De R$ 998,01 até R$ 1.497,00.',
                                                                   'De R$ 1.497,01 até R$ 1.996,00.',
                                                                   'De R$ 1.996,01 até R$ 2.495,00.',
                                                                   'De R$ 2.495,01 até R$ 2.994,00.'],
                    'De 3 Salários Mínimos até 5 Salários Mínimos': ['De R$ 2.994,01 até R$ 3.992,00.',
                                                                     'De R$ 3.992,01 até R$ 4.990,00.'],
                    'De 5 Salários Mínimos até 10 Salários Mínimos': ['De R$ 4.990,01 até R$ 5.988,00.',
                                                                      'De R$ 5.988,01 até R$ 6.986,00.',
                                                                      'De R$ 6.986,01 até R$ 7.984,00.',
                                                                      'De R$ 7.984,01 até R$ 8.982,00.',
                                                                      'De R$ 8.982,01 até R$ 9.980,00.'],
                    'De 10 Salários Mínimos para cima': ['De R$ 9.980,01 até R$ 11.976,00.',
                                                         'De R$ 11.976,01 até R$ 14.970,00.',
                                                         'De R$ 14.970,01 até R$ 19.960,00.',
                                                         'Mais de R$ 19.960,00.']},
            '2020':{'Nenhuma Renda': ['Nenhuma Renda'],
                    'Até 1 salário mínimo': ['Até R$ 1.045,00'],
                    'De 1 Salário Mínimo até 3 Salários Mínimos': ['De R$ 1.045,01 até R$ 1.567,50',
                                                                   'De R$ 1.567,51 até R$ 2.090,00',
                                                                   'R$ 2.090,01 até R$ 2.612,50', 
                                                                   'De R$ 2.612,51 até R$ 3.135,00'],
                    'De 3 Salários Mínimos até 5 Salários Mínimos': ['De R$ 3.135,01 até R$ 4.180,00',
                                                                     'De R$ 4.180,01 até R$ 5.225,00'],
                    'De 5 Salários Mínimos até 10 Salários Mínimos': ['De R$ 5.225,01 até R$ 6.270,00',
                                                                      'De R$ 6.270,01 até R$ 7.315,00',
                                                                      'De R$ 7.315,01 até R$ 8.360,00',
                                                                      'De R$ 8.360,01 até R$ 9.405,00',
                                                                      'De R$ 9.405,01 até R$ 10.450,00'],
                    'De 10 Salários Mínimos para cima': ['De R$ 10.450,01 até R$ 12.540,00',
                                                         'De R$ 12.540,01 até R$ 15.675,00',
                                                         'De R$ 15.675,01 até R$ 20.900,00',
                                                         'Acima de R$ 20.900,00']},
            '2021':{'Nenhuma Renda': ['Nenhuma Renda'],
                    'Até 1 salário mínimo': ['Até R$ 1.100,00'],
                    'De 1 Salário Mínimo até 3 Salários Mínimos': ['De R$ 1.100,01 até R$ 1.650,00.',
                                                                   'De R$ 1.650,01 até R$ 2.200,00.',
                                                                   'De R$ 2.200,01 até R$ 2.750,00.',
                                                                   'De R$ 2.750,01 até R$ 3.300,00.'],
                    'De 3 Salários Mínimos até 5 Salários Mínimos': ['De R$ 3.300,01 até R$ 4.400,00.',
                                                                     'De R$ 4.400,01 até R$ 5.500,00.'],
                    'De 5 Salários Mínimos até 10 Salários Mínimos': ['De R$ 5.500,01 até R$ 6.600,00.',
                                                                      'De R$ 6.600,01 até R$ 7.700,00.',
                                                                      'De R$ 7.700,01 até R$ 8.800,00.',
                                                                      'De R$ 8.800,01 até R$ 9.900,00.',
                                                                      'De R$ 9.900,01 até R$ 11.000,00.'],
                    'De 10 Salários Mínimos para cima': ['De R$ 11.000,01 até R$ 13.200,00.',
                                                         'De R$ 13.200,01 até R$ 16.500,00.',
                                                         'De R$ 16.500,01 até R$ 22.000,00.',
                                                         'Acima de R$ 22.000,00.']}

In [36]:
# Unificação de categorias de renda através dos anos
dp_renda = {'2019':{'Nenhuma renda.':'Nenhuma Renda',
                    'Até R$ 998,00.':'Até 1 salário mínimo',
                    'De R$ 998,01 até R$ 1.497,00.':'De 1 Salário Mínimo até 3 Salários Mínimos',
                    'De R$ 1.497,01 até R$ 1.996,00.':'De 1 Salário Mínimo até 3 Salários Mínimos',
                    'De R$ 1.996,01 até R$ 2.495,00.':'De 1 Salário Mínimo até 3 Salários Mínimos',
                    'De R$ 2.495,01 até R$ 2.994,00.':'De 1 Salário Mínimo até 3 Salários Mínimos',
                    'De R$ 2.994,01 até R$ 3.992,00.':'De 3 Salários Mínimos até 5 Salários Mínimos',
                    'De R$ 3.992,01 até R$ 4.990,00.':'De 3 Salários Mínimos até 5 Salários Mínimos',
                    'De R$ 4.990,01 até R$ 5.988,00.':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 5.988,01 até R$ 6.986,00.':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 6.986,01 até R$ 7.984,00.':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 7.984,01 até R$ 8.982,00.':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 8.982,01 até R$ 9.980,00.':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 9.980,01 até R$ 11.976,00.':'De 10 Salários Mínimos para cima',
                    'De R$ 11.976,01 até R$ 14.970,00.':'De 10 Salários Mínimos para cima',
                    'De R$ 14.970,01 até R$ 19.960,00.':'De 10 Salários Mínimos para cima',
                    'Mais de R$ 19.960,00.':'De 10 Salários Mínimos para cima'},
            '2020':{'Nenhuma Renda':'Nenhuma Renda',
                    'Até R$ 1.045,00':'Até 1 salário mínimo',
                    'De R$ 1.045,01 até R$ 1.567,50':'De 1 Salário Mínimo até 3 Salários Mínimos',
                    'De R$ 1.567,51 até R$ 2.090,00':'De 1 Salário Mínimo até 3 Salários Mínimos',
                    'De R$ 2.090,01 até R$ 2.612,50':'De 1 Salário Mínimo até 3 Salários Mínimos',
                    'De R$ 2.612,51 até R$ 3.135,00':'De 1 Salário Mínimo até 3 Salários Mínimos',
                    'De R$ 3.135,01 até R$ 4.180,00':'De 3 Salários Mínimos até 5 Salários Mínimos',
                    'De R$ 4.180,01 até R$ 5.225,00':'De 3 Salários Mínimos até 5 Salários Mínimos',
                    'De R$ 5.225,01 até R$ 6.270,00':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 6.270,01 até R$ 7.315,00':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 7.315,01 até R$ 8.360,00':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 8.360,01 até R$ 9.405,00':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 9.405,01 até R$ 10.450,00':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 10.450,01 até R$ 12.540,00':'De 10 Salários Mínimos para cima',
                    'De R$ 12.540,01 até R$ 15.675,00':'De 10 Salários Mínimos para cima',
                    'De R$ 15.675,01 até R$ 20.900,00':'De 10 Salários Mínimos para cima',
                    'Acima de R$ 20.900,00':'De 10 Salários Mínimos para cima'},
            '2021':{'Nenhuma Renda':'Nenhuma Renda',
                    'Até R$ 1.100,00':'Até 1 salário mínimo',
                    'De R$ 1.100,01 até R$ 1.650,00.':'De 1 Salário Mínimo até 3 Salários Mínimos',
                    'De R$ 1.650,01 até R$ 2.200,00.':'De 1 Salário Mínimo até 3 Salários Mínimos',
                    'De R$ 2.200,01 até R$ 2.750,00.':'De 1 Salário Mínimo até 3 Salários Mínimos',
                    'De R$ 2.750,01 até R$ 3.300,00.':'De 1 Salário Mínimo até 3 Salários Mínimos',
                    'De R$ 3.300,01 até R$ 4.400,00.':'De 3 Salários Mínimos até 5 Salários Mínimos',
                    'De R$ 4.400,01 até R$ 5.500,00.':'De 3 Salários Mínimos até 5 Salários Mínimos',
                    'De R$ 5.500,01 até R$ 6.600,00.':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 6.600,01 até R$ 7.700,00.':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 7.700,01 até R$ 8.800,00.':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 8.800,01 até R$ 9.900,00.':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 9.900,01 até R$ 11.000,00.':'De 5 Salários Mínimos até 10 Salários Mínimos',
                    'De R$ 11.000,01 até R$ 13.200,00.':'De 10 Salários Mínimos para cima',
                    'De R$ 13.200,01 até R$ 16.500,00.':'De 10 Salários Mínimos para cima',
                    'De R$ 16.500,01 até R$ 22.000,00.':'De 10 Salários Mínimos para cima',
                    'Acima de R$ 22.000,00.':'De 10 Salários Mínimos para cima'}
           }

In [37]:
#df_bkp = df_enem.copy()
#df_enem = df_bkp.copy()

In [38]:
# Criar nova coluna
for ano, cols_lista in dp_renda.items():
    # Gerar condições
    c0 = df_enem['NU_ANO'] == int(ano)
    # Aplicar
    df_enem.loc[c0, 'RENDA_FAMILIAR'] = df_enem.loc[c0, 'Q006'].replace(cols_lista)

**Derrubar colunas não necessárias**

In [42]:
# Drop
df_enem.drop(columns=['CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT'], inplace=True)

**Criar nova coluna - `NOTA_FINAL_PESO_UFBA_EE` & `NOTA_FINAL_PESO_UFBA_MED` & `NOTA_CORTE_EE` & `NOTA_CORTE_MED`**

- [Engenharia Elétrica](https://sisusimulator.com.br/ufba/campus-federacao-ondina/engenharia-eletrica-bacharelado)
- 2019 - LC - 2 | CH - 2 | CN - 4 | MT 4 | RD - 3 | NOTA DE CORTE 19.1 - 767.04
- 2020 - LC - 2 | CH - 2 | CN - 4 | MT 4 | RD - 3 | NOTA DE CORTE 20.1 - 749.28
- 2021 - LC - 2 | CH - 2 | CN - 4 | MT 4 | RD - 3 | NOTA DE CORTE 21.1 - 731.33

- [Medicina](https://sisusimulator.com.br/ufba/unidades-dispersas-salvador-(faculdade-de-medicina)/medicina-bacharelado)
- 2019 - LC - 2 | CH - 2 | CN - 4 | MT 4 | RD - 3 | NOTA DE CORTE 19.1 - 789.46
- 2020 - LC - 3 | CH - 3 | CN - 4 | MT 2 | RD - 3 | NOTA DE CORTE 20.1 - 773.07
- 2021 - LC - 2 | CH - 2 | CN - 4 | MT 2 | RD - 3 | NOTA DE CORTE 21.1 - 794.17

In [43]:
def calcular_nota_ufba_ssa(nota_lc, nota_ch, nota_cn, nota_mt, nota_rd, p_lc, p_ch, p_cn, p_mt, p_rd):
    '''
    Função para calcular a nota do aluno para a ufba
    '''
    # Calcular média ponderada
    mp_num = np.nansum([nota_lc*p_lc, nota_ch*p_ch, nota_cn*p_cn, nota_mt*p_mt, nota_rd*p_rd])
    mp_den = p_lc + p_ch + p_cn + p_mt + p_rd
    # Retornar
    return mp_num / mp_den

In [44]:
# Calcular EE
df_enem['NOTA_FINAL_PESO_UFBA_EE'] = df_enem.apply(lambda df:calcular_nota_ufba_ssa(df['NU_NOTA_LC'],
                                                                                    df['NU_NOTA_CH'],
                                                                                    df['NU_NOTA_CN'],
                                                                                    df['NU_NOTA_MT'],
                                                                                    df['NU_NOTA_REDACAO'],
                                                                                    2,2,4,4,3), axis=1)
# Calcular MED
df_enem['NOTA_FINAL_PESO_UFBA_MED'] = df_enem.apply(lambda df:calcular_nota_ufba_ssa(df['NU_NOTA_LC'],
                                                                                     df['NU_NOTA_CH'],
                                                                                     df['NU_NOTA_CN'],
                                                                                     df['NU_NOTA_MT'],
                                                                                     df['NU_NOTA_REDACAO'],
                                                                                     2,2,4,2,3), axis=1)

In [45]:
# Criar nota de corte - EE
df_enem.loc[df_enem['NU_ANO']==2019, 'NOTA_CORTE_EE'] = 767.04
df_enem.loc[df_enem['NU_ANO']==2020, 'NOTA_CORTE_EE'] = 749.28
df_enem.loc[df_enem['NU_ANO']==2021, 'NOTA_CORTE_EE'] = 731.33

# Criar nota de corte - MED
df_enem.loc[df_enem['NU_ANO']==2019, 'NOTA_CORTE_MED'] = 789.46
df_enem.loc[df_enem['NU_ANO']==2020, 'NOTA_CORTE_MED'] = 773.07
df_enem.loc[df_enem['NU_ANO']==2021, 'NOTA_CORTE_MED'] = 794.17

In [46]:
# Ordenar dataframe final para salvar
df_enem.sort_values(['NU_ANO','NOTA_FINAL_PESO_UFBA_MED'], ascending=False, inplace=True)

# Exportar dados para análise

In [47]:
df_enem.to_csv('dados/transformacao/dados_enem_19_20_21.csv', index=False)